In [1]:
!pip install -q sentence-transformers faiss-cpu

zsh:1: /Users/hannahjoshua/Desktop/PRINC OF SOFT/CBT/cbt_env/bin/pip: bad interpreter: /Users/hannahjoshua/cbt_env/bin/python3: no such file or directory

[notice] A new release of pip is available: 24.1.2 -> 25.2
[notice] To update, run: python3.10 -m pip install --upgrade pip


In [ ]:
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np

In [ ]:

thought_response_pairs = [
    ("I'm a failure.", "Everyone fails sometimes. What matters is learning and growing."),
    ("Nobody likes me.", "Some people might not understand me yet, and that's okay."),
    ("I'll never get better.", "Progress takes time. I'm already improving by trying."),
    ("I always mess things up.", "Mistakes happen, but they help me improve."),
    ("I'm not good enough.", "I am doing my best, and that is enough."),
    ("I can't do this.", "It's okay to struggle. I can take it one step at a time."),
    ("I'm weak for feeling this way.", "Emotions don't make me weak. They make me human."),
    ("There's no point in trying.", "Every small effort counts. Change is possible."),
]

#dataset